In [1]:
from gurobipy import *
from itertools import combinations, product

In [2]:
m = 3
n = 4

arcs = []
for layer in range(m-1):
    for i in range(n):
        for j in range(n):
            arcs.append((n*layer+i+1 , n*(layer+1)+j+1))

print(arcs)

[(1, 5), (1, 6), (1, 7), (1, 8), (2, 5), (2, 6), (2, 7), (2, 8), (3, 5), (3, 6), (3, 7), (3, 8), (4, 5), (4, 6), (4, 7), (4, 8), (5, 9), (5, 10), (5, 11), (5, 12), (6, 9), (6, 10), (6, 11), (6, 12), (7, 9), (7, 10), (7, 11), (7, 12), (8, 9), (8, 10), (8, 11), (8, 12)]


In [3]:
def testsys(not_shortest):
    LP = Model()
    LP.Params.LogToConsole = 0
    x = LP.addVars(arcs, vtype=GRB.CONTINUOUS, name="x")
    for a in arcs:
        LP.addConstr(x[a] >= 0)
    for i in range(n):
        for j in range(n):
            for m1 in range(n):
                for m2 in range(m1 + 1, n):
                    a = i + 1
                    c = 2 * n + j + 1
                    b1 = n + m1 + 1
                    b2 = n + m2 + 1
                    if (a, b1, c) in not_shortest and (a, b2, c) in not_shortest:
                        continue;
                    elif (a, b1, c) in not_shortest:
                        LP.addConstr(x[(a, b2)] + x[(b2, c)] + 1 <= x[(a, b1)] + x[(b1, c)])
                    elif (a, b2, c) in not_shortest:
                        LP.addConstr(x[(a, b1)] + x[(b1, c)] + 1 <= x[(a, b2)] + x[(b2, c)])
                    else:
                        LP.addConstr(x[(a, b1)] + x[(b1, c)] == x[(a, b2)] + x[(b2, c)])
    LP.setObjective(quicksum(x[a] for a in arcs))
    LP.optimize()
    #obj = LP.getObjective()
    return LP.status
    
#not_shortest = [(1, 4, 7), (2, 5, 8), (3, 6, 9)]
not_shortest = [(1, 4, 7), (1, 4, 8), (1, 4, 9)]
testsys(not_shortest)

Using license file C:\Users\Lily W\gurobi.lic
Academic license - for non-commercial use only


2

In [4]:
allpaths = []

for i in range(n):
    for j in range(n):
        for k in range(n):
            a = i + 1
            b = n + j + 1
            c = 2 * n + k + 1
            allpaths.append((a, b, c))
print(allpaths)

[(1, 5, 9), (1, 5, 10), (1, 5, 11), (1, 5, 12), (1, 6, 9), (1, 6, 10), (1, 6, 11), (1, 6, 12), (1, 7, 9), (1, 7, 10), (1, 7, 11), (1, 7, 12), (1, 8, 9), (1, 8, 10), (1, 8, 11), (1, 8, 12), (2, 5, 9), (2, 5, 10), (2, 5, 11), (2, 5, 12), (2, 6, 9), (2, 6, 10), (2, 6, 11), (2, 6, 12), (2, 7, 9), (2, 7, 10), (2, 7, 11), (2, 7, 12), (2, 8, 9), (2, 8, 10), (2, 8, 11), (2, 8, 12), (3, 5, 9), (3, 5, 10), (3, 5, 11), (3, 5, 12), (3, 6, 9), (3, 6, 10), (3, 6, 11), (3, 6, 12), (3, 7, 9), (3, 7, 10), (3, 7, 11), (3, 7, 12), (3, 8, 9), (3, 8, 10), (3, 8, 11), (3, 8, 12), (4, 5, 9), (4, 5, 10), (4, 5, 11), (4, 5, 12), (4, 6, 9), (4, 6, 10), (4, 6, 11), (4, 6, 12), (4, 7, 9), (4, 7, 10), (4, 7, 11), (4, 7, 12), (4, 8, 9), (4, 8, 10), (4, 8, 11), (4, 8, 12)]


In [5]:
l = len(allpaths)


In [6]:
list(combinations([1,2, 3], 2))
minimal = []


#fixed = [i*m+1 for i in range(n)]
fixed = (1, 5, 9)
for a in range(1, 7):
    allpathsys = list(combinations(range(1, l), a))
    for s in allpathsys:
        notshortest = [fixed]
        is_minimal = True
        for i in s:
            notshortest.append(allpaths[i])
        for mi in minimal:
            if set(mi).issubset(set(notshortest)):
                is_minimal = False
                break
        if not is_minimal:
            continue
        if testsys(notshortest) == 2:
            print(notshortest)
            minimal.append(notshortest)
        

[(1, 5, 9), (1, 5, 10), (1, 5, 11), (1, 5, 12)]
[(1, 5, 9), (1, 6, 9), (1, 7, 9), (1, 8, 9)]
[(1, 5, 9), (2, 5, 9), (3, 5, 9), (4, 5, 9)]


KeyboardInterrupt: 

In [ ]:
#hardcoded for now
print(range(m))
layer1 = [i + 1 for i in range(m)]
layer2 = [m + i + 1 for  i in range(m)]
layer3 = [2*m + i + 1 for i in range(m)]
t1 = list(combinations(layer1, 2))
t2 = list(combinations(layer2, 2))
t3 = list(combinations(layer3, 2))
subgraphst = list(product(t1, t2, t3))
subgraphs = []
for t in subgraphst:
    tlist = t[0]+t[1]+t[2]
    subgraphs.append(tlist)

def detect_forbidden_minor(notshortest):
    for s in subgraphs:
        contained = []
        for p in notshortest:
            if set(p).issubset(set(s)):
                contained.append(p)
        if len(contained) == 1:
            return [1, s]
        elif len(contained) <= 4 and len(contained) >= 2:
            pairs = list(combinations(contained, 2))
            forbidden = True
            for p in pairs:
                overlap = set(p[0]).intersection(set(p[1]))
                if len(overlap) != 1:
                    forbidden = False
                    break
            if forbidden:
                return [2, s]
    return [0, []]

    #check type 1:

    

In [ ]:
minimal = []

fixed = (1, 5, 9)
for a in range(1, 7):
    allpathsys = list(combinations(range(1, l), a))
    for s in allpathsys:
        notshortest = [fixed]
        is_minimal = True
        for i in s:
            notshortest.append(allpaths[i])
        for m in minimal:
            if set(m).issubset(set(notshortest)):
                is_minimal = False
                break
        if not is_minimal:
            continue
        out = detect_forbidden_minor(notshortest)
        if out[0] == 0:
            print(notshortest)
            minimal.append(notshortest)